In [1]:
from preprocessing import LabeledSentencesDataset, GenerateWordEmbeddings
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, Subset
from torchvision import transforms, utils
import numpy as np

In [2]:
yelp_train_0 = LabeledSentencesDataset(0, "../data/yelp/sentiment.train.0", "../data/yelp/",
                        transform=GenerateWordEmbeddings(200, "../data/glove.6B/"))
yelp_train_1 = LabeledSentencesDataset(1, "../data/yelp/sentiment.train.1", "../data/yelp/",
                        transform=GenerateWordEmbeddings(200, "../data/glove.6B/"))


In [2]:
yelp_train_0 = LabeledSentencesDataset(0, "../data/yelp/sentiment.train.0", "../data/yelp/",
                        transform=None)
yelp_train_1 = LabeledSentencesDataset(1, "../data/yelp/sentiment.train.1", "../data/yelp/",
                        transform=None)

In [3]:
batch_size = 32

full_train = ConcatDataset([yelp_train_0, yelp_train_1])

size_of_pretrain = 128
pretrain = Subset(full_train, range(0,size_of_pretrain))
train = Subset(full_train, range(size_of_pretrain,len(full_train)))

pretrain_loader = DataLoader(pretrain, batch_size=batch_size, shuffle=True)
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)

In [4]:
count = 0
for thing in train_loader:
    count += 1
print(count)

13875


In [5]:

sample = "Run Away!"
tf = GenerateWordEmbeddings(50, "../data/glove.6B/")
len(tf(sample))

NameError: name 'dim' is not defined

In [ ]:
a = torch.randn(1, 3, 6, 8)
p2d = (1, 1, 2, 2) # pad last dim by (1, 1) and 2nd to last by (2, 2)
F.pad(a, p2d, 'constant', 0)